In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
#Task3
train_data=pd.read_csv('train.tsv',sep='\t')
valid_data=pd.read_csv('valid.tsv',sep='\t')

train_data['label'].replace({"Lit-News_mentions":0, "Nonpersonal_reports":1, "Self_reports":2}, inplace=True)
valid_data['label'].replace({"Lit-News_mentions":0, "Nonpersonal_reports":1, "Self_reports":2}, inplace=True)

X_train=train_data['tweet']
y_train=train_data['label']
X_test=valid_data['tweet']
y_test=valid_data['label']

doc2vec with svm

In [3]:
from bs4 import BeautifulSoup
import re
def cleanText(text):
    text = BeautifulSoup(text, "lxml").text
    text = re.sub(r'\|\|\|', r' ', text) 
    text = re.sub(r'http\S+', r'<URL>', text)
    text = text.lower()
    text = text.replace('x', '')
    return text
train_data['tweet'] = train_data['tweet'].apply(cleanText)
valid_data['tweet'] = valid_data['tweet'].apply(cleanText)


In [4]:
from gensim.models.doc2vec import TaggedDocument
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens
#train_tagged=train_data['tweet'].apply(tokenize_text)
#valid_tagged=valid_data['tweet'].apply(tokenize_text)

train_tagged = train_data.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['tweet']), tags=[r.label]), axis=1)
test_tagged = valid_data.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['tweet']), tags=[r.label]), axis=1)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
import multiprocessing
cores = multiprocessing.cpu_count()

In [6]:
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, hs=0, min_count=2, sample = 0, workers=cores)
model_dbow.build_vocab([x for x in tqdm(train_tagged)])

100%|██████████| 9067/9067 [00:00<00:00, 1240815.50it/s]


In [7]:
train_tagged.values

array([TaggedDocument(words=['growing', 'number', 'of', 'covid-19', 'patients', 'whose', 'symptoms', 'were', 'initially', 'mild', 'are', 'now', 'facing', 'mysterious', 'long-term', 'neurological', 'problems', 'url'], tags=[0]),
       TaggedDocument(words=['medical', 'eperts', 'advise', 'that', 'symptoms', 'of', 'the', 'novel', 'coronavirus', 'include', 'fever', 'shortness', 'of', 'breath', 'and', 'stinky', 'smelly', 'pits', 'and', 'feet', '😳🤪'], tags=[0]),
       TaggedDocument(words=['drdavidsamadi', 'hubby/i', 'same', 'symptoms', 'november', '2019', 'after', 'weekend', 'trip', 'vegas', 'where', 'bus', 'loads', 'of', 'chinese', 'tourists.1', 'day', 'fever,3', 'days', 'sore', 'throat', 'several', 'weeks', 'of', 'fatigue.he', "'s", 'healthy', "'m", 'not', 'diabetes', 'hypertension', 'obese', 'respiratory', 'issues', '53.', 'no', 'meds/pneumonia', 'we', 'believe', 'was', 'covid-19'], tags=[1]),
       ...,
       TaggedDocument(words=['yes', 'but', 'didn', 'recovered', 'with', 'tylenol'

In [8]:
%%time
from sklearn import utils
for epoch in range(30):
    model_dbow.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), total_examples=len(train_tagged.values), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

100%|██████████| 9067/9067 [00:00<00:00, 1036882.91it/s]


CPU times: user 39.3 s, sys: 2.32 s, total: 41.6 s
Wall time: 24.1 s


In [9]:
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, regressors

In [10]:
from sklearn.svm import SVC
y_train, X_train = vec_for_learning(model_dbow, train_tagged)
y_test, X_test = vec_for_learning(model_dbow, test_tagged)
svcClassifier=SVC(kernel='rbf')
svcClassifier.fit(X_train, y_train)
y_pred = svcClassifier.predict(X_test)
from sklearn.metrics import accuracy_score, f1_score
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred,average='micro')))

Testing accuracy 0.572
Testing F1 score: 0.572
